In [1]:
%pip install -qU langchain-openai langchain langgraph langchain_community langgraph-supervisor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 9.3 MB/s eta 0:00:00


In [2]:
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from google.colab import userdata
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor
from langchain_core.tools import tool
from langchain_core.messages import (
    HumanMessage,
)
from langgraph.prebuilt import create_react_agent
import requests
key = userdata.get('OPENAI_API_KEY')

In [3]:
model = ChatOpenAI(api_key=key,model="gpt-4o-mini")

# Tool Define

In [ ]:
from langchain.tools import tool
from typing import List
import requests

# Optional: Use a dataclass to format results more clearly
def get_spotify_token(client_id: str, client_secret: str) -> str:
    """
    Fetches a bearer token from Spotify using client credentials.
    """
    url = 'https://accounts.spotify.com/api/token'
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    data = {'grant_type': 'client_credentials'}
    response = requests.post(url, headers=headers, data=data, auth=(client_id, client_secret))
    response.raise_for_status()
    return response.json()['access_token']


@tool
def search_spotify_playlists(query: str) -> str:
    """
    Search for Spotify playlists using a keyword (e.g., "sleep", "meditation").

    Returns the top 5 playlist names with their Spotify links.
    """

    # Replace with your actual credentials (or store securely)
    client_id = "api id"
    client_secret = "api secret"

    try:
        token = get_spotify_token(client_id, client_secret)


        headers = {
            'Authorization': f'Bearer {token}'
        }
        params = {
            'q': query,
            'type': 'playlist',
            'limit': 5
        }
        url = 'https://api.spotify.com/v1/search'
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()

        playlists = response.json().get('playlists', {}).get('items', [])

        if not playlists:
            return "No playlists found for your query."
        result = []
        for pl in playlists:
            if pl is None:
                continue  # Skip null entries

            name = pl.get('name', 'Unknown Title')
            link = pl.get('external_urls', {}).get('spotify', 'No URL available')
            desc = pl.get('description', 'None description')

            result.append(f"🎵 {name}\n🔗 {link} 📝 {desc.strip()[:100] or 'No description.'}")

        return "\n\n".join(result)

    except Exception as e:
        return f"Failed to search playlists: {e}"



# Tool Run

In [ ]:
search_spotify_playlists.run("happy")

''

In [18]:
search_spotify_playlists.run("meditation")

'🎵 Tibetan Healing Music, Zen Meditation, Stress Relief, Relaxing Music, Tibetan Bowls, Indian Flute\n🔗 https://open.spotify.com/playlist/0mnWqdNjnmb0AqNmXzQ0Vz 📝 Tibetan Healing Sounds - Cleanse the aura and space. Indian Flute Meditation Music, Yoga Music, Posi'

In [5]:
search_spotify_playlists.run("meditation")

'🎵 Tibetan Healing Music, Zen Meditation, Stress Relief, Relaxing Music, Tibetan Bowls, Indian Flute\n🔗 https://open.spotify.com/playlist/0mnWqdNjnmb0AqNmXzQ0Vz\n📝 Tibetan Healing Sounds - Cleanse the aura and space. Indian Flute Meditation Music, Yoga Music, Posi'

In [6]:
search_spotify_playlists.run("sleep")

'🎵 Fall into Sleep Instantly\n🔗 https://open.spotify.com/playlist/6L0keGopfWZMQ76K4O8bX1\n📝 No description.\n\n🎵 Thunderstorm Sounds - 10 Hours for Sleeping\n🔗 https://open.spotify.com/playlist/6wx6zQaEaQ7pWj7e0CIM3S\n📝 I created this Thunderstorm Sounds Playlist for Relaxing Sleep, Deep Sleep and ASMR Sleep. It&#x27;s'

# Agent Definition

In [19]:
spotify_assistant = create_react_agent(
    model=model,
    tools=[search_spotify_playlists],
    prompt="You are a helpful Spotify assistant. Your job is to help users discover Spotify playlists based on their search queries. ",
    name="spotify_assistant"
)


# Agent Invoke

In [20]:
response = spotify_assistant.invoke(
    {
        "messages": [
            HumanMessage(content="happy")
            ]
    }
    )
response["messages"]

[HumanMessage(content='happy', additional_kwargs={}, response_metadata={}, id='cbac463b-e9b9-4c26-9cc8-02870fb4a5eb'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_BoK12RjePquH2Xaxmh0XQC7l', 'function': {'arguments': '{"query":"happy"}', 'name': 'search_spotify_playlists'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 95, 'total_tokens': 112, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BhCR4VU9rc6tnbsJXyZ8CBm6GBwm9', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='spotify_assistant', id='run--562ada7d-45bf-4d7a-917e-e4af30092596-0', tool_calls=[{'name': 'search_spotify_playlists', 'args': {'query': 'happ

In [21]:
response = spotify_assistant.invoke(
    {
        "messages": [
            HumanMessage(content="I can't sleep well.")
            ]
    }
    )
response["messages"]

[HumanMessage(content="I can't sleep well.", additional_kwargs={}, response_metadata={}, id='1b02db02-5ac2-49c4-9d32-595912562056'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_oxkxCaHlncazUU5I1xlzcEEl', 'function': {'arguments': '{"query":"sleep"}', 'name': 'search_spotify_playlists'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 99, 'total_tokens': 116, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BhCRAB07gzFEy5g7rrpM5XqTE8osu', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='spotify_assistant', id='run--a62db029-a74d-489f-8d14-8486c65a7c8a-0', tool_calls=[{'name': 'search_spotify_playlists', 'args': {

In [22]:
response["messages"][-1].content

'Here are some Spotify playlists that might help you sleep better:\n\n1. **[Fall into Sleep Instantly](https://open.spotify.com/playlist/6L0keGopfWZMQ76K4O8bX1)** - No description available.\n2. **[Thunderstorm Sounds - 10 Hours for Sleeping](https://open.spotify.com/playlist/6wx6zQaEaQ7pWj7e0CIM3S)** - A playlist for relaxing sleep, deep sleep, and ASMR sleep.\n\nTry listening to these playlists to help improve your sleep quality!'